In [1]:
!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

from keras import backend as K
K.clear_session()

import warnings
warnings.filterwarnings('ignore')

import re
import scipy
import pandas as pd
import io
import numpy as np
import copy
import torch
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import class_weight
from torch import nn, optim
from torch.utils import data

from sklearn.model_selection import train_test_split 

In [2]:
#Seeding for deterministic results
RANDOM_SEED = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
HIDDEN_LAYER_UNITS = 128
# CLASS_NAMES = ['biased','neutral']
EPOCHS      = 3

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# # Use (small) WIKIBIAS-Manual

# train_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wikibias/train.tsv", sep='\t', header=None, index_col=False)
# dev_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wikibias/dev.tsv", sep='\t', header=None, index_col=False)
# test_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wikibias/test.tsv", sep='\t', header=None, index_col=False)

# # WIKIBIAS Preprocess

# col = {0:"sentences", 1:"label0", 2:"label" }

# train_df = train_df.rename(columns = col)
# dev_df = dev_df.rename(columns = col)
# test_df = test_df.rename(columns = col)

# train_df = train_df.drop(columns=['label0'])
# dev_df = dev_df.drop(columns=['label0'])
# test_df = test_df.drop(columns=['label0'])

# X_train=train_df["sentences"]
# X_test=test_df["sentences"]
# X_dev=dev_df["sentences"]

# y_train=train_df["label"]
# y_test=test_df["label"]
# y_dev=dev_df["label"]


# # Use (full) WIKIBIAS-Auto

# # large_train_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wikibias/noisy_train.tsv", sep='\t', header=None, index_col=False)
# # large_train_df = large_train_df.rename(columns = col)
# # large_train_df = large_train_df.drop(columns=['label0'])
# # X, test_df, y, y_test = train_test_split (large_train_df, large_train_df.label.values, test_size=0.2, train_size=0.8 )
# # train_df, dev_df, y_train, y_dev = train_test_split(X, y, test_size = 0.2, train_size =0.8)

# # class weight (not implemented): check for which (WB-manual or WB-auto) dataset is required
# # class_weights = class_weight.compute_class_weight(
# #                                         class_weight = "balanced",
# #                                         classes = np.unique(y_train),
# #                                         y = y_train                                                    
# #                                     )
# # print(class_weights)

In [5]:
# WNC (180k each for 0-1 labels)
# # biased and neutral not in pairs in custom DF(Sambit)

train_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wnc_corpus/wnc/biased.word.train", sep='\t', header=None, index_col=False)
dev_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wnc_corpus/wnc/biased.word.dev", sep='\t', header=None, index_col=False)
test_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wnc_corpus/wnc/biased.word.test", sep='\t', header=None, index_col=False)

large_train_df=pd.read_csv(F"/content/gdrive/My Drive/SAMBIT/master_thesis/data/wnc_corpus/wnc/biased.full", sep='\t', header=None, index_col=False)

col = {0:"id", 1:"src_tok", 2:"tgt_tok", 3:"src_raw", 4:"tgt_raw", 5:"src_POS_tags", 6:"tgt_parse_tags" }
col_neutral = {0:"id", 1:"src_tok", 2:"tgt_tok", 3:"src_raw", 4:"tgt_raw"}

train_df = train_df.rename(columns = col)
dev_df = dev_df.rename(columns = col)
test_df = test_df.rename(columns = col)

# neutral_df = neutral_df.rename(columns = col_neutral)
large_train_df = large_train_df.rename(columns = col)



In [6]:
large_train_df.head(2)

,id,src_tok,tgt_tok,src_raw,tgt_raw,src_POS_tags,tgt_parse_tags
0,258378316,"during the campaign , controversy erupted over...","during the campaign , some pointed out alleged...","during the campaign, controversy erupted over ...","during the campaign, some pointed out alleged ...",ADP DET NOUN PUNCT NOUN VERB ADP VERB NOUN ADP...,prep det pobj punct nsubj ROOT prep amod pobj ...
1,486527143,nic ##aea was con ##vo ##ked by the emperor co...,nic ##aea was con ##vo ##ked by the emperor co...,nicaea was convoked by the emperor constantine...,nicaea was convoked by the emperor constantine...,NOUN NOUN VERB VERB VERB VERB ADP DET NOUN NOU...,nsubjpass nsubjpass auxpass ROOT ROOT ROOT age...


In [8]:

# preprocess larger_train_df
biased_df = pd.DataFrame()
neutral_df = pd.DataFrame()
biased_df["sentences"] = large_train_df["src_raw"]
biased_df["label"] = 1
neutral_df["sentences"] = large_train_df["tgt_raw"]
neutral_df["label"] = 0

wnc_df = pd.concat([biased_df, neutral_df])
wnc_df = wnc_df.reset_index()
wnc_df = wnc_df.drop(columns=['index'])

from sklearn.model_selection import train_test_split 

X, test_df, y, y_test = train_test_split (wnc_df, wnc_df.label.values, test_size=0.2, train_size=0.8 )
train_df, dev_df, y_train, y_dev = train_test_split(X, y, test_size = 0.2, train_size =0.8)

In [9]:
print(train_df.count())
print(test_df.count())
print(dev_df.count())

sentences    232313
label        232313
dtype: int64
sentences    72598
label        72598
dtype: int64
sentences    58079
label        58079
dtype: int64


In [10]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    dev_df.sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    test_df.sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [12]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(dev_df.label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']


In [13]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test)

In [14]:
# encoded_data_train
# input_ids_train


In [16]:
label_dict={'0': 0, '1': 1}
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cuda


In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [19]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
#     return f1_score(labels_flat, preds_flat, average='weighted')
    return f1_score(labels_flat, preds_flat, average=None)

def confusion_matrix_output(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(confusion_matrix(labels_flat,preds_flat))

def f1_score_test(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(label_dict_inverse[label], ':F1 score =',f1_score(y_true, y_preds,average=None))
#         print(label_dict_inverse[label], ': Weighted F1 score =',f1_score(y_true, y_preds,average='weighted'))

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Statistics: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
        print(f'Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')

In [20]:
import random
from tqdm.notebook import tqdm

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    # torch.save(model.state_dict(), f'/content/gdrive/My Drive/SAMBIT/master_thesis/results/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/7260 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5858646051901759
Validation loss: 0.5586592222048231
F1 Score (Weighted): [0.70727935 0.68585911]


Epoch 2:   0%|          | 0/7260 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5298101605873135
Validation loss: 0.5651202591325954
F1 Score (Weighted): [0.72343101 0.66523311]


Epoch 3:   0%|          | 0/7260 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.49294421056454835
Validation loss: 0.5917001742633577
F1 Score (Weighted): [0.71145898 0.67585551]


# validation

In [21]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(label_dict),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)

# model.to(device)

# model.load_state_dict(torch.load('/content/gdrive/My Drive/SAMBIT/master_thesis/results/finetuned_BERT_epoch_5.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
test_f1 = f1_score_test(predictions, true_vals)
accuracy_per_class(predictions, true_vals)
# print('\n')

confusion_matrix_output(predictions,true_vals)

0 :F1 score = [0.85761362 0.        ]
1 :F1 score = [0.        0.7792602]
Class: 0
Statistics: 21861/29120

Accuracy: 0.7507211538461539

Class: 1
Statistics: 18486/28959

Accuracy: 0.6383507717807936

[[21861  7259]
 [10473 18486]]


In [22]:
torch.save(model.state_dict(), f'/content/gdrive/My Drive/SAMBIT/master_thesis/results/finetuned_BERT_WNC_large.model')
        

In [23]:
# from torchinfo import summary
repr(model)
sum([param.nelement() for param in model.parameters()])


109483778

# test

In [24]:
def predict(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
#                   'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
#         loss = outputs[0]
        logits = outputs[0]
#         loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
#         label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
#         true_vals.append(label_ids)
    
#     loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
#     true_vals = np.concatenate(true_vals, axis=0)   
#     return loss_val_avg, predictions, true_vals
    return predictions

In [25]:
prediction_test = predict(dataloader_test)
df_predict=pd.DataFrame()
label_dict_inverse = {v: k for k, v in label_dict.items()}
preds_flat = np.argmax(prediction_test, axis=1).flatten()
# print(label_dict_inverse[preds_flat[0]])
for i in range(0,preds_flat.size):
    temp=[]
    temp.append(label_dict_inverse[preds_flat[i]])
    df_predict=df_predict.append(temp)

In [26]:
df_predict.to_excel('/content/gdrive/My Drive/SAMBIT/master_thesis/results/prediction_bert_v1.xlsx')
# df_predict.head(10)

In [27]:
df_predict.head(10)

,0
0,0
0,0
0,0
0,1
0,0
0,1
0,1
0,0
0,0
0,1
